<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [64]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer


### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [65]:
times = ['time%s' % i for i in range(1, 11)]

In [66]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('test_sessions.csv',
                      index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на заголовок обучающей выборки
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [67]:
test_df.head(20)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44
6,1218,2014-12-05 11:21:52,1220.0,2014-12-05 11:21:52,1202.0,2014-12-05 11:21:52,1221.0,2014-12-05 11:21:52,1222.0,2014-12-05 11:21:52,302.0,2014-12-05 11:21:52,1217.0,2014-12-05 11:21:53,1216.0,2014-12-05 11:21:53,1221.0,2014-12-05 11:21:53,1220.0,2014-12-05 11:21:53
7,43986,2014-05-21 14:10:43,43986.0,2014-05-21 14:10:44,43986.0,2014-05-21 14:10:45,43986.0,2014-05-21 14:10:46,43986.0,2014-05-21 14:11:01,43986.0,2014-05-21 14:11:04,43986.0,2014-05-21 14:11:05,43986.0,2014-05-21 14:11:06,43986.0,2014-05-21 14:11:07,43986.0,2014-05-21 14:11:08
8,3346,2014-05-25 18:32:04,3346.0,2014-05-25 18:33:59,23.0,2014-05-25 18:34:03,3346.0,2014-05-25 18:34:03,3359.0,2014-05-25 18:34:03,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
9,1062,2014-06-05 10:25:52,21.0,2014-06-05 10:25:52,23.0,2014-06-05 10:25:52,22.0,2014-06-05 10:25:52,11009.0,2014-06-05 10:25:53,8220.0,2014-06-05 10:25:53,1063.0,2014-06-05 10:25:53,21.0,2014-06-05 10:25:53,2896.0,2014-06-05 10:25:54,292.0,2014-06-05 10:25:54


В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [68]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# загрузим словарик сайтов
with open(r"site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)
site_dict
# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [69]:
# наша целевая переменная
y_train = train_df['target']

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]


Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [70]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [71]:
full_sites.to_csv('data_sessions_text.txt', sep=' ', index=None, header=None)

In [72]:
cv = CountVectorizer()
with open('data_sessions_text.txt') as inp_file:
    data = cv.fit_transform(inp_file)
data = csr_matrix(data)

In [73]:
X_train_sparse = data[:idx_split, :]
X_test_sparse = data[idx_split:, :]

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [74]:
def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    idx = round(X.shape[0] * ratio)
    lr = LogisticRegression(C = C, random_state = seed)
    X_train = X[:idx, :]
    y_train = y[:idx]
    X_valid = X[idx:, :]
    y_valid = y[idx:]
    lr.fit(X_train, y_train)
    y_pred = lr.predict_proba(X_valid)[:, 1]
    score = roc_auc_score(y_valid, y_pred)
    return score

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [75]:
get_auc_lr_valid(X_train_sparse, y_train)

0.9166808568709574

Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [76]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [77]:
lr = LogisticRegression(C = 1, random_state = 17)
lr.fit(X_train_sparse, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=17, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [78]:
y_pred = lr.predict_proba(X_test_sparse)[:, 1]

In [79]:
write_to_submission_file(y_pred, "submit.csv")

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [80]:
train_df["time1"].apply(lambda ts: int(ts.year*100 + ts.month))

session_id
21669     201301
54843     201301
77292     201301
114021    201301
146670    201301
           ...  
12224     201404
164438    201404
12221     201404
156968    201404
204762    201404
Name: time1, Length: 253561, dtype: int64

In [81]:
new_feat_train = pd.DataFrame(index = train_df.index)
new_feat_test = pd.DataFrame(index = test_df.index)
new_feat_train["year_month"] = train_df["time1"].apply(lambda ts: int(ts.year*100 + ts.month))
new_feat_test["year_month"] = test_df["time1"].apply(lambda ts: int(ts.year*100 + ts.month))

Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [82]:
scaler = StandardScaler()
scaler.fit(new_feat_train["year_month"].values.reshape(-1, 1))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [83]:
new_feat_train["year_month_scales"] = scaler.transform(new_feat_train["year_month"].values.reshape(-1, 1))
new_feat_test["year_month_scales"] = scaler.transform(new_feat_test["year_month"].values.reshape(-1, 1))

In [84]:
X_train_sparse_new = csr_matrix(hstack([X_train_sparse, new_feat_train["year_month_scales"].values.reshape(-1, 1)]))

In [85]:
get_auc_lr_valid(X_train_sparse_new, y_train)

0.9163173461057248

**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [86]:
new_feat_train["start_hour"] = train_df["time1"].apply(lambda x: x.hour)
new_feat_test["start_hour"] = test_df["time1"].apply(lambda x: x.hour)
new_feat_train["morning"] = [1 if x.hour <= 11 else 0 for x in train_df["time1"]]
new_feat_test["morning"] = [1 if x.hour <= 11 else 0 for x in test_df["time1"]]

In [87]:
new_feat_train.head()

,year_month,year_month_scales,start_hour,morning
session_id,,,,
21669,201301,-1.744405,8,1
54843,201301,-1.744405,8,1
77292,201301,-1.744405,8,1
114021,201301,-1.744405,8,1
146670,201301,-1.744405,8,1


In [88]:
#start_month & start_hour

In [89]:
month_hour_train = new_feat_train[["year_month_scales", "start_hour"]]

In [90]:
X_train_month_hour = csr_matrix(hstack([X_train_sparse, month_hour_train.values.reshape(-1, 2)]))

In [91]:
get_auc_lr_valid(X_train_month_hour, y_train)

0.9557888334565163

In [92]:
month_hour_test = new_feat_test[["year_month_scales", "start_hour"]]

In [93]:
X_test_month_hour = csr_matrix(hstack([X_test_sparse, month_hour_test.values.reshape(-1, 2)]))

In [94]:
lr.fit(X_train_month_hour, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=17, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [95]:
y_pred_month_hour = lr.predict_proba(X_test_month_hour)[:, 1]

In [96]:
write_to_submission_file(y_pred_month_hour, "month_hour_submit_1.csv")

In [97]:
#start_month & morning

In [98]:
month_morning_train = new_feat_train[["year_month_scales", "morning"]]

In [99]:
X_train_month_morning = csr_matrix(hstack([X_train_sparse, month_morning_train.values.reshape(-1, 2)]))

In [100]:
get_auc_lr_valid(X_train_month_morning, y_train)

0.9468007731547904

In [101]:
month_morning_test = new_feat_test[["year_month_scales", "morning"]]

In [102]:
X_test_month_morning = csr_matrix(hstack([X_test_sparse, month_morning_test.values.reshape(-1, 2)]))

In [103]:
lr.fit(X_train_month_morning, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=17, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [104]:
y_pred_month_morn = lr.predict_proba(X_test_month_morning)[:, 1]

In [105]:
write_to_submission_file(y_pred_month_morn, "month_morning_submit_1.csv")

In [106]:
#start_month & morning & start_hour

In [107]:
month_morning_hour_train = new_feat_train[["year_month_scales", "morning", "start_hour"]]

In [108]:
X_train_month_hour_morning = csr_matrix(hstack([X_train_sparse, month_morning_hour_train.values.reshape(-1, 3)]))

In [109]:
get_auc_lr_valid(X_train_month_hour_morning, y_train)

0.9568559670110961

In [110]:
month_morning_hour_test = new_feat_test[["year_month_scales", "morning", "start_hour"]]

In [111]:
X_test_month_morning_hour = csr_matrix(hstack([X_test_sparse, month_morning_hour_test.values.reshape(-1, 3)]))

In [112]:
lr.fit(X_train_month_hour_morning, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=17, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [113]:
y_pred_month_hour_morn = lr.predict_proba(X_test_month_morning_hour)[:, 1]

In [114]:
write_to_submission_file(y_pred_month_hour_morn, "month_morning_hour_submit_1.csv")

### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [115]:
get_auc_lr_valid(X_train_month_morning, y_train)

0.9468007731547904

Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [116]:
def get_auc_lr_valid(X, y, C, ratio = 0.9, seed=17):
    idx = round(X.shape[0] * ratio)
    lr = LogisticRegression(C = C, random_state = seed)
    X_train = X[:idx, :]
    y_train = y[:idx]
    X_valid = X[idx:, :]
    y_valid = y[idx:]
    lr.fit(X_train, y_train)
    y_pred = lr.predict_proba(X_valid)[:, 1]
    score = roc_auc_score(y_valid, y_pred)
    return score

In [117]:
dic = {}

In [118]:
for C in np.logspace(-3, 1, 10):
    dic[C] = get_auc_lr_valid(X_train_month_morning, y_train, C)  

In [119]:
C_val = max(dic, key=dic.get)

In [120]:
lr_best = LogisticRegression(C = C_val)

In [121]:
lr_best.fit(X_train_month_morning, y_train)

LogisticRegression(C=0.1668100537200059, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [122]:
y_pred_month_morn = lr_best.predict_proba(X_test_month_morning)[:, 1]

In [123]:
write_to_submission_file(y_pred_month_morn, "month_morning_submit_3.csv")

In [124]:
!telegram-send "Досчитал"